In [ ]:
import os
from pathlib import Path

# This snippet ensures consistent import paths across environments.
# When running notebooks via JupyterLab's web UI, the current working
# directory is often different (e.g., /notebooks) compared to VS Code,
# which typically starts at the project root. This handles that by 
# retrying the import after changing to the parent directory.
# 
# Include this at the top of every notebook to standardize imports
# across development environments.

try:
    from utils.os import chdir_to_git_root
except ModuleNotFoundError:
    os.chdir(Path.cwd().parent)
    print(f"Retrying import from: {os.getcwd()}")
    from utils.os import chdir_to_git_root

chdir_to_git_root("python")

print(os.getcwd())

In [ ]:
import logging
from models.pytorch.narrative_stack.stage1.preprocessing import extract_concept_unit_value_tuples, get_valid_concepts, collect_concept_unit_pairs, generate_concept_unit_embeddings, generate_concepts_report
from db import DB

db = DB()
data_dir = "../data/june-us-gaap" # Where CSV data is read from (once CSV file per symbol)

In [ ]:
logging.info("Collecting valid concepts...")
valid_concepts = get_valid_concepts(db)

logging.info("Extracting concept unit value tuples...")
extracted_concept_unit_value_data = extract_concept_unit_value_tuples(data_dir, valid_concepts)

In [ ]:
# # For deterministic hashing (TODO: Move to tests)

# import hashlib
# import pickle

# # def hash_extracted_data(data: ExtractedConceptUnitValueData) -> str:
# def hash_extracted_data(data) -> str:
#     """
#     Computes a SHA-256 hash of the full extracted concept/unit/value data structure.
#     This includes tuples, unit stats, and file list — all serialized deterministically.
#     """
#     # Serialize using protocol=5 (highest and deterministic in modern Python)
#     serialized = pickle.dumps(data.dict(), protocol=5)
#     return hashlib.sha256(serialized).hexdigest()

# hash_extracted_data(extracted_concept_unit_value_data)

In [ ]:
# print("Fetching...")
# extracted_concept_unit_value_data_2 = extract_concept_unit_value_tuples(data_dir, valid_concepts)

# print("Hashing...")
# hash_extracted_data(extracted_concept_unit_value_data_2)

In [ ]:
# View concepts report (not needed for preprocessing but contains useful information)

generate_concepts_report(extracted_concept_unit_value_data)

In [ ]:
from utils.pytorch import get_device, seed_everything

device = get_device()

logging.info("Collecting concept unit pairs...")
concept_unit_pairs = collect_concept_unit_pairs(extracted_concept_unit_value_data)

logging.info(f"Total concept unit pairs: {len(concept_unit_pairs)}")

logging.info("Generating concept unit embeddings...")
concept_unit_embeddings = generate_concept_unit_embeddings(concept_unit_pairs, device=device)
# concept_unit_embeddings_2 = generate_concept_unit_embeddings(concept_unit_pairs, device=device)



In [ ]:
# import numpy as np
# import torch
# from tqdm import tqdm

# # Normalize for cosine similarity
# A = concept_unit_embeddings_1
# B = concept_unit_embeddings_2

# A_norm = A / np.linalg.norm(A, axis=1, keepdims=True)
# B_norm = B / np.linalg.norm(B, axis=1, keepdims=True)

# # Cosine similarity per row
# cos_sim = np.sum(A_norm * B_norm, axis=1)

# # Report
# print(f"Cosine similarity:")
# print(f"  Mean: {cos_sim.mean():.8f}")
# print(f"  Min:  {cos_sim.min():.8f}")
# print(f"  Std:  {cos_sim.std():.8f}")

# # Optional: show rows below threshold
# threshold = 0.999
# bad_indices = np.where(cos_sim < threshold)[0]
# print(f"\n🔻 Below {threshold}: {len(bad_indices)} / {len(cos_sim)} rows")
# if len(bad_indices):
#     for idx in bad_indices[:10]:
#         print(f"  Row {idx}: cosine = {cos_sim[idx]:.8f}")


In [ ]:
# concept_unit_embeddings

In [ ]:
from models.pytorch.narrative_stack.stage1.preprocessing.plots import plot_pca_explanation

dim = plot_pca_explanation(concept_unit_embeddings, variance_threshold=0.95)

display(dim)

In [ ]:
from models.pytorch.narrative_stack.stage1.preprocessing import pca_compress_concept_unit_embeddings

# TODO: Capture PCA, and store the instance directly instead of refitting
pca_compressed_concept_unit_embeddings, _ = pca_compress_concept_unit_embeddings(concept_unit_embeddings, n_components=243)
# pca_compressed_concept_unit_embeddings_2, _ = pca_compress_concept_unit_embeddings(concept_unit_embeddings_2, n_components=243)

In [ ]:
# import numpy as np
# import torch
# from tqdm import tqdm

# # Run embedding twice
# # embeddings_3 = generate_concept_unit_embeddings(concept_unit_pairs, device=device)
# # embeddings_4 = generate_concept_unit_embeddings(concept_unit_pairs, device=device)

# # assert embeddings_1.shape == embeddings_2.shape, "Shape mismatch"

# # Normalize for cosine similarity
# A = pca_compressed_concept_unit_embeddings
# B = pca_compressed_concept_unit_embeddings_2

# A_norm = A / np.linalg.norm(A, axis=1, keepdims=True)
# B_norm = B / np.linalg.norm(B, axis=1, keepdims=True)

# # Cosine similarity per row
# cos_sim = np.sum(A_norm * B_norm, axis=1)

# # Report
# print(f"Cosine similarity:")
# print(f"  Mean: {cos_sim.mean():.8f}")
# print(f"  Min:  {cos_sim.min():.8f}")
# print(f"  Std:  {cos_sim.std():.8f}")

# # Optional: show rows below threshold
# threshold = 0.999
# bad_indices = np.where(cos_sim < threshold)[0]
# print(f"\n🔻 Below {threshold}: {len(bad_indices)} / {len(cos_sim)} rows")
# if len(bad_indices):
#     for idx in bad_indices[:10]:
#         print(f"  Row {idx}: cosine = {cos_sim[idx]:.8f}")


In [ ]:
from models.pytorch.narrative_stack.stage1.preprocessing.plots import plot_semantic_embeddings

plot_semantic_embeddings(pca_compressed_concept_unit_embeddings, title="PCA Semantic Embedding Scatterplot")
plot_semantic_embeddings(concept_unit_embeddings, title="Raw Semantic Embedding Scatterplot")

In [ ]:
pca_compressed_concept_unit_embeddings

In [ ]:
import logging
import numpy as np

# TODO: Add types
def save_concept_unit_value_tuples(pca_compressed_concept_unit_embeddings, concept_unit_pairs, concept_unit_value_tuples, file_path):
    assert len(pca_compressed_concept_unit_embeddings) == len(concept_unit_pairs), \
        f"Mismatch: {len(pca_compressed_concept_unit_embeddings)} embeddings vs {len(concept_unit_pairs)} keys"

    # Save both embeddings and tuples
    np.savez_compressed(
        file_path,
        keys=np.array([f"{c}::{u}" for c, u in concept_unit_pairs]),
        embeddings=pca_compressed_concept_unit_embeddings,
        concept_unit_value_tuples=np.array(concept_unit_value_tuples, dtype=object)
    )

    logging.info(f"Saved {len(concept_unit_value_tuples):,} tuples and {len(pca_compressed_concept_unit_embeddings):,} embeddings to '{file_path}'")


save_concept_unit_value_tuples(
    pca_compressed_concept_unit_embeddings,
    concept_unit_pairs,
    extracted_concept_unit_value_data.concept_unit_value_tuples,
    "data/stage1_latents.npz"
)

# save_concept_unit_value_tuples(
#     pca_compressed_concept_unit_embeddings_2,
#     concept_unit_pairs,
#     extracted_concept_unit_value_data.concept_unit_value_tuples,
#     "data/stage1_latents_new_2.npz"
# )

In [ ]:
# import numpy as np

# # Load from disk
# new_data = np.load("data/stage1_latents_new.npz", allow_pickle=True)
# new_concept_unit_value_tuples = new_data["concept_unit_value_tuples"].tolist()
# new_embeddings = new_data["embeddings"]

# # Check shape match
# assert len(pca_compressed_concept_unit_embeddings) == len(new_embeddings), \
#     "Mismatch in embedding row counts"

# # Cosine similarity check
# def cosine_similarity(a, b):
#     a = a / np.linalg.norm(a)
#     b = b / np.linalg.norm(b)
#     return np.dot(a, b)

# cos_sims = []
# for a_vec, b_vec in zip(pca_compressed_concept_unit_embeddings, new_embeddings):
#     sim = cosine_similarity(a_vec.astype(np.float64), b_vec.astype(np.float64))
#     cos_sims.append(sim)

# # Report
# cos_sims = np.array(cos_sims)
# print(f"✅ Compared {len(cos_sims)} rows")
# print(f"🔹 Mean cosine similarity: {cos_sims.mean():.8f}")
# print(f"🔹 Min cosine similarity:  {cos_sims.min():.8f}")
# print(f"🔹 Std dev:                {cos_sims.std():.8f}")


In [ ]:
# import numpy as np

# # Load saved latent data
# old_data = np.load("data/stage1_latents.npz", allow_pickle=True)

# # Build embedding map
# embedding_map = {
#     tuple(key.split("::", 1)): vec
#     for key, vec in zip(old_data["keys"], old_data["embeddings"])
# }

# # Load concept-unit-value tuples
# old_concept_unit_value_tuples = old_data["concept_unit_value_tuples"].tolist()

# # Load saved latent data
# new_data = np.load("data/stage1_latents_new.npz", allow_pickle=True)

# # Build embedding map
# embedding_map = {
#     tuple(key.split("::", 1)): vec
#     for key, vec in zip(new_data["keys"], new_data["embeddings"])
# }

# # Load concept-unit-value tuples
# new_concept_unit_value_tuples = new_data["concept_unit_value_tuples"].tolist()

In [ ]:
# import numpy as np
# from hashlib import sha256

# # a = np.load("data/stage1_latents.npz", allow_pickle=True)
# a = np.load("data/stage1_latents_new_1.npz", allow_pickle=True)
# b = np.load("data/stage1_latents_new_2.npz", allow_pickle=True)

# # with open("data/stage1_latents_new_1.pkl", "rb") as f:
# #     a = pickle.load(f)

# # with open("data/stage1_latents_new_2.pkl", "rb") as f:
# #     b = pickle.load(f)

# def hash_array(arr):
#     return sha256(np.ascontiguousarray(arr)).hexdigest()

# print("Hash a[keys]:", hash_array(a["keys"]))
# print("Hash b[keys]:", hash_array(b["keys"]))

# for k in a.files:
# # for k in a.keys():
#     print(f"Checking: {k}")
#     if k == "embeddings":
#         A = a[k].astype(np.float32)
#         B = b[k].astype(np.float32)
#         assert A.shape == B.shape, "Shape mismatch in embeddings"

#         # Normalize to unit vectors
#         A_norm = A / np.linalg.norm(A, axis=1, keepdims=True)
#         B_norm = B / np.linalg.norm(B, axis=1, keepdims=True)

#         # Cosine similarity
#         cos_sim = np.sum(A_norm * B_norm, axis=1)
#         mean_sim = np.mean(cos_sim)
#         min_sim = np.min(cos_sim)

#         print(f"Mean cosine similarity: {mean_sim:.8f}")
#         print(f"Min cosine similarity:  {min_sim:.8f}")
#         print(f"Std cosine similarity:  {cos_sim.std():.8f}")
#         assert min_sim > 0.999, f"Cosine similarity too low in embeddings: {min_sim}"
#     else:
#         assert np.array_equal(a[k], b[k]), f"Mismatch in {k}"
